In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Dropout, Masking, Bidirectional
from keras.regularizers import l2, l1
import numpy as np
from readdata import read_data
import collections

x_train, y_train, x_text, y_test = read_data(level=0, length_limit=750)
# x_train = x_train * 5
# y_train = y_train * 5

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
targets = set()
for i, (input_text, target) in enumerate(zip(x_train, y_train)):
    input_texts.append(input_text)
    target_texts.append(target)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    if target not in targets:
        targets.add(target)

input_characters = sorted(list(input_characters))
targets = sorted(list(targets))
num_encoder_tokens = len(input_characters)
nb_targets = len(targets)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output classes:', nb_targets)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
print('Class counts:', [(key, value) for
                        key, value in (collections.Counter(y_train)).iteritems()])

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(targets)])

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_index = dict(
    (i, char) for char, i in target_token_index.items())



encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
y_encoded = np.zeros((len(input_texts), nb_targets), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, -(t+1), input_token_index[char]] = 1.
    y_encoded[i, target_token_index[target_text]] = 1.



batch_size = 16  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 16  # Latent dimensionality of the encoding space.

encoder_inputs = Input(shape=(None, num_encoder_tokens))
mask_1 = Masking(mask_value=0.0)(encoder_inputs)
encoder_1 = Bidirectional(LSTM(latent_dim, return_sequences=True,  
                               W_regularizer=l2(0.004)))(mask_1)
encoder_2 = Bidirectional(LSTM(latent_dim, return_sequences=False,  
                               W_regularizer=l2(0.004)))(encoder_1)
# encoder_2 = LSTM(latent_dim*2,  W_regularizer=l2(0.001), recurrent_dropout=0.3)(encoder_1)
# encoder_states = [state_h, state_c]
dense_1 = Dense(64, activation='relu')(encoder_2)
dropout_1 = Dropout(0.50)(dense_1)
# dense_2 = Dense(128, activation='relu')(dropout_1)
# dropout_2 = Dropout(0.50)(dense_1)
dense_outputs = Dense(nb_targets, activation='softmax')(dropout_1)

model = Model(encoder_inputs, dense_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(encoder_input_data, y_encoded,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          verbose=1)

a = 5
print "end"

Using TensorFlow backend.


('Number of samples:', 8134)
('Number of unique input tokens:', 21)
('Number of unique output classes:', 6)
('Max sequence length for inputs:', 750)
('Max sequence length for outputs:', 1)
('Class counts:', [('1', 1596), ('3', 2225), ('2', 1660), ('5', 539), ('4', 1033), ('6', 1081)])


/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:75: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, return_sequences=True, kernel_regularizer=<keras.reg...)`
/home/acar/.virtualenvs/gpuenv/lib/python2.7/site-packages/ipykernel_launcher.py:77: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(16, return_sequences=False, kernel_regularizer=<keras.reg...)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 21)          0         
_________________________________________________________________
masking_1 (Masking)          (None, None, 21)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 32)          4864      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                6272      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total para